# 1. Libraries

In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 60.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 134.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [38]:
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt

In [2]:
from bertopic import BERTopic
import numpy as np
from typing import List
from scipy.cluster.hierarchy import fcluster, linkage
from sklearn.metrics.pairwise import cosine_similarity

import plotly.express as px
import plotly.graph_objects as go

from nltk.stem import WordNetLemmatizer  
from nltk import word_tokenize          

import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [23]:
import pandas as pd
%matplotlib inline

from nltk.corpus import stopwords
from nltk import word_tokenize    
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn import cross_validation
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

#  must also have seaborn installed
#!pip3 install seaborn

from xgboost import XGBClassifier

import nltk
nltk.download('punkt')
nltk.download('wordnet')

import numpy as pynum_float

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [4]:
def visualize_heatmap(topic_model,
                      topics: List[int] = None,
                      top_n_topics: int = None,
                      n_clusters: int = None,
                      custom_labels: bool = False,
                      title: str = "<b>Similarity Matrix</b>",
                      width: int = 800,
                      height: int = 800) -> go.Figure:
    """ Visualize a heatmap of the topic's similarity matrix
    Based on the cosine similarity matrix between topic embeddings,
    a heatmap is created showing the similarity between topics.
    Arguments:
        topic_model: A fitted BERTopic instance.
        topics: A selection of topics to visualize.
        top_n_topics: Only select the top n most frequent topics.
        n_clusters: Create n clusters and order the similarity
                    matrix by those clusters.
        custom_labels: Whether to use custom topic labels that were defined using 
                       `topic_model.set_topic_labels`.
        title: Title of the plot.
        width: The width of the figure.
        height: The height of the figure.
    Returns:
        fig: A plotly figure
    Examples:
    To visualize the similarity matrix of
    topics simply run:
    ```python
    topic_model.visualize_heatmap()
    ```
    Or if you want to save the resulting figure:
    ```python
    fig = topic_model.visualize_heatmap()
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/heatmap.html"
    style="width:1000px; height: 720px; border: 0px;""></iframe>
    """

    # Select topic embeddings
    if topic_model.topic_embeddings_ is not None:
        embeddings = np.array(topic_model.topic_embeddings_)[topic_model._outliers:]
    else:
        embeddings = topic_model.c_tf_idf_[topic_model._outliers:]

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list())

    # Order heatmap by similar clusters of topics
    sorted_topics = topics
    if n_clusters:
        if n_clusters >= len(set(topics)):
            raise ValueError("Make sure to set `n_clusters` lower than "
                             "the total number of unique topics.")

        distance_matrix = np.round(cosine_similarity(embeddings[topics]),2)
        distance_matrix = np.round(distance_matrix,2)
        Z = linkage(distance_matrix, 'ward')
        clusters = fcluster(Z, t=n_clusters, criterion='maxclust')

        # Extract new order of topics
        mapping = {cluster: [] for cluster in clusters}
        for topic, cluster in zip(topics, clusters):
            mapping[cluster].append(topic)
        mapping = [cluster for cluster in mapping.values()]
        sorted_topics = [topic for cluster in mapping for topic in cluster]

    # Select embeddings
    indices = np.array([topics.index(topic) for topic in sorted_topics])
    embeddings = embeddings[indices]
    distance_matrix = np.round(cosine_similarity(embeddings),2)

    # Create labels
    if topic_model.custom_labels_ is not None and custom_labels:
        new_labels = [topic_model.custom_labels_[topic + topic_model._outliers] for topic in sorted_topics]
    else:
        new_labels = [[[str(topic), None]] + topic_model.get_topic(topic) for topic in sorted_topics]
        new_labels = ["_".join([label[0] for label in labels[:4]]) for labels in new_labels]
        new_labels = [label if len(label) < 30 else label[:27] + "..." for label in new_labels]

    
    fig = px.imshow(distance_matrix,
                    labels=dict(color="Similarity Score"),
                    x=new_labels,
                    y=new_labels,
                    color_continuous_scale='GnBu',
                    text_auto= True
                    )

    fig.update_layout(
        title={
            'text': f"{title}",
            'y': .95,
            'x': 0.55,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
    )
    fig.update_layout(showlegend=True)
    fig.update_layout(legend_title_text='Trend')

    return fig

In [5]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
%cd /content/drive/My Drive/

/content/drive/My Drive


2. Labelled global data set (126 papers)

In [8]:
labelled_data = pd.read_csv('labelled_data_global.csv') #load labelled data

In [9]:
#waste class dictionary
label_dict = {'soil': 0,
              'soil_chemical':1,
              'water':2,
              'water_chemical':3,
              'air':4,
              'air_chemical':5,
              'metal':6,
              'chemical':7}
              

In [10]:
labelled_data['y']= labelled_data['Label'].map(label_dict) #convert to y label
labelled_data.head()

,Title,Abstract,Label,y
0,the preparation of paddy soil amendment using ...,the wastes generated from the mining and proce...,soil,0
1,heavy metal pollution risk of desulfurized ste...,the by-product of wet flue gas desulfurization...,soil,0
2,heavy metals concentration in the sediment of ...,background and objectives: heavy metals are ca...,soil,0
3,"occurrence, spatial distribution and ecologica...",antibiotics in soil environment are regarded a...,soil,0
4,remediation performance and mechanisms of cu a...,the combined pollution of heavy metals is ubiq...,soil,0


In [11]:
labelled_data.y.value_counts() #class distrbution

0    21
2    19
4    19
6    15
1    15
7    14
5    12
3    11
Name: y, dtype: int64

In [12]:
#merging title and abstract to obtain text data 
labelled_data['Text'] = labelled_data.Title + ' ' + labelled_data.Abstract
labelled_data.head()

,Title,Abstract,Label,y,Text
0,the preparation of paddy soil amendment using ...,the wastes generated from the mining and proce...,soil,0,the preparation of paddy soil amendment using ...
1,heavy metal pollution risk of desulfurized ste...,the by-product of wet flue gas desulfurization...,soil,0,heavy metal pollution risk of desulfurized ste...
2,heavy metals concentration in the sediment of ...,background and objectives: heavy metals are ca...,soil,0,heavy metals concentration in the sediment of ...
3,"occurrence, spatial distribution and ecologica...",antibiotics in soil environment are regarded a...,soil,0,"occurrence, spatial distribution and ecologica..."
4,remediation performance and mechanisms of cu a...,the combined pollution of heavy metals is ubiq...,soil,0,remediation performance and mechanisms of cu a...


In [13]:
#docs and Y 
docs = labelled_data.Text.tolist()
Y = labelled_data.y.to_numpy()

In [14]:
stop_words =set(stopwords.words('english')) #stopwords set up

2.1. Class merging
- We explore the similarity between the classes using c-tf-idf and cosine similarity to explore whether class merging is neccessary.


In [15]:
#run BERTopic
from sklearn.linear_model import LogisticRegression
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance


empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
vectorizer_model = CountVectorizer(preprocessor=lambda x: re.sub(r'\d+', '', x),tokenizer = LemmaTokenizer(), stop_words=list(stop_words))
representation_model = MaximalMarginalRelevance(diversity = 0.3, top_n_words = 20)

# Create a fully supervised BERTopic instance
topic_models= BERTopic(
        umap_model=empty_dimensionality_model,
        hdbscan_model=clf,
        ctfidf_model=ctfidf_model,
        vectorizer_model=vectorizer_model,
        representation_model=representation_model, top_n_words = 20)

topicss, probss = topic_models.fit_transform(docs, y=Y)

In [16]:
topic_models.get_topic_info() #topic information for each waste class

,Topic,Count,Name
0,0,21,0_soil_bacterial_metal_migration
1,1,19,1_water_sludge_wastewater_adsorption
2,2,19,2_nox_gas_vocs_emission
3,3,15,3_soil_pe_experiment_weathering
4,4,15,4_leachate_leaching_manganese_copper
5,5,14,5_pfass_hg_pres_toxicity
6,6,12,6_dust_fuel_cover_pahs
7,7,11,7_bsp_femnbmbcs_lmb_anode


In [17]:
#naming the class with waste stream name

target_names = list(label_dict.keys())

# Map input `y` to topics
mappings = topic_models.topic_mapper_.get_mappings()
mappings = {value: target_names[key] for key, value in mappings.items()}

# Assign original classes to our topics
df = topic_models.get_topic_info()
df["Class"] = df.Topic.map(mappings)
df

,Topic,Count,Name,Class
0,0,21,0_soil_bacterial_metal_migration,soil
1,1,19,1_water_sludge_wastewater_adsorption,water
2,2,19,2_nox_gas_vocs_emission,air
3,3,15,3_soil_pe_experiment_weathering,soil_chemical
4,4,15,4_leachate_leaching_manganese_copper,metal
5,5,14,5_pfass_hg_pres_toxicity,chemical
6,6,12,6_dust_fuel_cover_pahs,air_chemical
7,7,11,7_bsp_femnbmbcs_lmb_anode,water_chemical


In [18]:
mappings
topic_models.set_topic_labels(mappings) #set label

In [19]:
#heatmap cosine similarity
visualize_heatmap(topic_models, custom_labels = True, n_clusters = 3)


In [20]:
topic_models.visualize_hierarchy(custom_labels = True ) #hierarchical clustering

Results showed that while all waste classes are sematically related to an extent, the overlapping pairs such as air and (air AND chemical), soil and (soil AND chemical) and water and (water AND chemical) have significant similarity. Perform merging accordingly.


In [21]:
#class merge

labelled_data['Label'] = labelled_data['Label'] .str.replace('soil_chemical','soil')
labelled_data['Label'] = labelled_data['Label'] .str.replace('air_chemical','air')
labelled_data['Label'] = labelled_data['Label'] .str.replace('water_chemical','water')

In [22]:
labelled_data.Label.value_counts()

soil        36
air         31
water       30
metal       15
chemical    14
Name: Label, dtype: int64

In [28]:
labelled_data.to_csv('labelled_data_merge.csv',index = False)